In [1]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import matplotlib as plot
import numpy as np

In [2]:
testData = pd.read_csv("./data/fashion-mnist_test.csv")


In [3]:
trainData = pd.read_csv("./data/fashion-mnist_train.csv")

In [4]:
labels=["T-Shirt/Top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [5]:
def array_map(function,x):
    return np.array(list(map(function, x)))

In [6]:
training_dataset = (
    tf.data.Dataset.from_tensor_slices(({
            "data":tf.cast(array_map(lambda y:y[1:785],testData.values), tf.float32),
            "label":tf.cast(tf.one_hot(array_map(lambda y: y[0],testData.values),depth=10),tf.int16),
        })
))
training_dataset=training_dataset.batch(50)
train_iter = training_dataset.make_initializable_iterator() # create the iterattor
train_el = train_iter.get_next()

In [7]:
test_dataset=(
    tf.data.Dataset.from_tensor_slices(
        {
            "data":tf.cast(array_map(lambda y:y[1:785],testData.values), tf.float32),
            "label":tf.cast(tf.one_hot(array_map(lambda y: y[0],testData.values),depth=10),tf.int16),
        }
    )
)
test_dataset=test_dataset.batch(300)
test_iter = test_dataset.make_initializable_iterator() # create the iterattor
test_el = test_iter.get_next()

In [8]:
writer = tf.summary.FileWriter("/tmp/mnist_fashion/1")

In [9]:
def init_weigths(shape):
    init_random_dist= tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist,name="W")

In [10]:
def init_bias(shape):
    init_bias_vals=tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals,name="B")

In [11]:
def conv2d(x,W):
    #x --> [batch,H,W,Channels]
    #W --> [filter H, filter W, Channels In, Channels Out]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")

In [12]:
def max_pool_2by2(x,name="max_pool_2by2"):
    with tf.name_scope(name):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [13]:
def convolutional_layer(input_x,shape, name="conv"):
    with tf.name_scope(name) : 
        W= init_weigths(shape)
        b= init_bias([shape[3]])
        return tf.nn.relu(conv2d(input_x,W)+b)

In [14]:
def normal_full_layer(input_layer, size,name="fc"):
    with tf.name_scope(name):
        
        input_size = int(input_layer.get_shape()[1])
        W=init_weigths([input_size,size])
        b=init_bias([size])
        return tf.matmul(input_layer,W) + b

In [15]:
x_input=tf.placeholder(tf.float32,shape=[None,784])
y_true= tf.placeholder(tf.int16,shape=[None,10])

In [16]:
x_image=tf.reshape(x_input,[-1,28,28,1])

In [17]:
convo_1= convolutional_layer(x_image,[5,5,1,32],"convo_1")
convo_1_pooling=max_pool_2by2(convo_1,"convo_1_2by2_maxPool")
convo_2=convolutional_layer(convo_1_pooling,[5,5,32,64],"convo_2")
convo_2_pooling=max_pool_2by2(convo_2,"convo_2_2by2_maxPool")

In [18]:

convo_2_flat=tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [19]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout= tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [20]:
y_pred= normal_full_layer(full_one_dropout,10)

In [21]:
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

In [25]:
optimizer= tf.train.AdamOptimizer(learning_rate=0.001)
train=optimizer.minimize(cross_entropy)

In [26]:
init=tf.global_variables_initializer()

In [27]:
steps = 5000
with tf.Session() as sess:
    writer.add_graph(sess.graph)
    sess.run(init)
    sess.run(train_iter.initializer)
    sess.run(test_iter.initializer)
    for i in range(steps):
        try :
            trainBatch=sess.run(train_el)
        except tf.errors.OutOfRangeError:
            print("exception")
            sess.run(train_iter.initializer)
            continue
        sess.run(train,feed_dict={x_input:trainBatch["data"],y_true:trainBatch["label"],hold_prob:0.7})
        if i%50 == 0:
            print("On Step {}".format(i))
            print("Accuracy")
            try :
                testBatch=sess.run(test_el)
            except tf.errors.OutOfRangeError:
                sess.run(test_iter.initializer)
                print("exception 2")
                continue
            matches=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc= tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc, feed_dict={x_input:testBatch["data"],y_true:testBatch["label"],hold_prob:1.0}))
            print('/n')

On Step 0
Accuracy
0.11666667
/n
On Step 50
Accuracy
0.78333336
/n
On Step 100
Accuracy
0.8433333
/n
On Step 150
Accuracy
0.8066667
/n
exception
On Step 250
Accuracy
0.8466667
/n
On Step 300
Accuracy
0.7966667
/n
On Step 350
Accuracy
0.83666664
/n
On Step 400
Accuracy
0.82
/n
exception
On Step 450
Accuracy
0.81666666
/n
On Step 500
Accuracy
0.9
/n
On Step 550
Accuracy
0.8433333
/n
On Step 600
Accuracy
0.86333334
/n
exception
On Step 650
Accuracy
0.86
/n
On Step 700
Accuracy
0.93
/n
On Step 750
Accuracy
0.89666665
/n
On Step 800
Accuracy
0.8933333
/n
exception
On Step 850
Accuracy
0.8466667
/n
On Step 900
Accuracy
0.8933333
/n
On Step 950
Accuracy
0.89
/n
On Step 1000
Accuracy
0.9066667
/n
exception
On Step 1050
Accuracy
0.9
/n
On Step 1100
Accuracy
0.89666665
/n
On Step 1150
Accuracy
0.9266667
/n
On Step 1200
Accuracy
0.9066667
/n
exception
On Step 1250
Accuracy
0.94
/n
On Step 1300
Accuracy
0.88
/n
On Step 1350
Accuracy
0.93666667
/n
On Step 1400
Accuracy
0.9433333
/n
exception
On Ste